In [1]:
# Answer a few questions before diving into the problem before  joining the features and response
# What do the features actually mean?
# Why don't the unique IDs of labels and telematic data match up?
# What does accuracy mean?
# What does second mean?
# What does acceleration and gyro actually mean?

In [2]:
import pandas as pd

In [3]:
df0 = pd.read_csv("../grab-ai-safety-data/features/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df1 = pd.read_csv("../grab-ai-safety-data/features/part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df2 = pd.read_csv("../grab-ai-safety-data/features/part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df3 = pd.read_csv("../grab-ai-safety-data/features/part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df4 = pd.read_csv("../grab-ai-safety-data/features/part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df5 = pd.read_csv("../grab-ai-safety-data/features/part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df6 = pd.read_csv("../grab-ai-safety-data/features/part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df7 = pd.read_csv("../grab-ai-safety-data/features/part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df8 = pd.read_csv("../grab-ai-safety-data/features/part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df9 = pd.read_csv("../grab-ai-safety-data/features/part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
response = pd.read_csv("../grab-ai-safety-data/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv")

In [4]:
df_features = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], axis=0)

In [5]:
df = pd.merge(
    df_features,
    response,
    how="left",
    on="bookingID"
)

In [15]:
# Let's look at a case where the booking is in fact dangerous
df.loc[df["bookingID"] == 1194000908341].sort_values("second")

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label
1891618,1194000908341,5.951,0.0,0.311845,9.574412,-0.573411,-0.011718,0.000000,-0.008522,0.0,0.00,1
316823,1194000908341,5.828,0.0,0.322020,9.548077,-0.548871,-0.011718,-0.006392,-0.006392,1.0,0.00,1
10174476,1194000908341,5.743,0.0,0.317232,9.557054,-0.590171,-0.008522,-0.004261,-0.005326,2.0,0.00,1
5281531,1194000908341,5.642,0.0,0.309451,9.547478,-0.636259,-0.009587,-0.006392,-0.007457,4.0,0.00,1
13264407,1194000908341,5.769,0.0,0.244209,9.701305,-0.053271,-0.011718,-0.005326,-0.006392,5.0,0.00,1
103145,1194000908341,5.867,0.0,0.181361,9.457695,-0.240019,-0.012783,-0.005326,-0.008522,6.0,0.00,1
5847195,1194000908341,5.941,0.0,0.181361,9.220071,-0.799663,-0.011718,-0.004261,-0.006392,7.0,0.00,1
1260580,1194000908341,5.996,0.0,0.380678,8.867524,-1.762132,-0.007457,-0.004261,-0.008522,8.0,0.00,1
1988713,1194000908341,6.038,0.0,0.084994,10.098743,0.742202,-0.014914,-0.004261,-0.007457,9.0,0.00,1
12985159,1194000908341,6.068,0.0,0.335787,9.545682,-0.545279,-0.009587,-0.005326,-0.007457,10.0,0.00,1


In [17]:
df.loc[df["bookingID"] == 	1099511627891].sort_values("second")

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label
5373013,1099511627891,3.900,292.0,1.168406,9.438231,3.395082,-0.066700,-0.028999,0.002840,0.0,14.916689,0
3240919,1099511627891,3.900,293.0,1.412622,8.439818,1.201926,0.026151,0.068129,0.024220,1.0,14.601616,0
13709919,1099511627891,3.900,295.0,0.442941,10.182851,4.189981,0.060970,-0.508528,0.217254,2.0,14.670257,0
3333692,1099511627891,3.900,296.0,0.026337,9.071907,3.028758,0.182533,-0.013727,-0.137048,3.0,14.324488,0
4491564,1099511627891,3.900,297.0,-0.138868,11.023241,1.534730,0.093346,-0.794413,0.147004,4.0,13.926226,0
2353861,1099511627891,3.900,298.0,0.751802,10.793391,4.431803,0.073188,-0.054655,-0.057636,6.0,13.518582,0
11748394,1099511627891,3.900,299.0,1.984854,10.903527,3.007209,-0.168104,0.575758,-0.108948,7.0,13.326422,0
11326732,1099511627891,3.900,299.0,-0.318439,8.614601,2.719896,-0.067922,-0.142009,-0.066188,8.0,12.979033,0
996121,1099511627891,3.900,300.0,2.552297,8.214757,0.754197,-0.232856,-0.283119,0.101800,9.0,12.699341,0
8609188,1099511627891,3.900,301.0,0.660820,9.763853,4.252232,-0.059981,-0.005175,0.090194,11.0,12.534326,0
